In [1]:
import torch
from transformers import AutoProcessor, MusicgenForConditionalGeneration

processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small").cuda()

/home/lego/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/lego/.local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [2]:
import torchaudio
wav, sr = torchaudio.load('test.wav')
wav = torchaudio.functional.resample(wav, sr, 32000)
wav = wav[None,:].repeat([2,1,1]).cuda()
print(wav.shape)

# inputs = processor(audio=wav.squeeze(), sampling_rate=32000,  return_tensors="pt",)
with torch.no_grad():
    output = model.audio_encoder.encode(
        input_values=wav,
        padding_mask=torch.ones_like(wav),
    )
    audio_codes = output['audio_codes']
    frames, bsz, codebooks, seq_len = audio_codes.shape
    decoder_input_ids = audio_codes[0, ...].reshape(bsz * codebooks, seq_len)
    decoder = model.decoder.get_decoder()

print('encode complete')
with torch.no_grad():
    decoder_outputs = decoder(
            input_ids=decoder_input_ids,
            attention_mask=None,
            encoder_attention_mask=None,
            encoder_hidden_states=None,
            head_mask=None,
            cross_attn_head_mask=None,
            past_key_values=None,
            inputs_embeds=None,
            use_cache=False,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=False,
        )

decoder_outputs
# frames, bsz, codebooks, seq_len = audio_codes.shape


torch.Size([2, 1, 1920000])
encode complete


(tensor([[[ 1.0411,  0.0592, -0.3752,  ...,  2.2531, -0.9834,  0.1631],
          [ 0.0884,  3.3004,  0.6958,  ...,  1.4224,  0.3608,  0.0968],
          [ 0.3378,  1.2210,  0.1548,  ..., -0.0409, -0.1516, -2.5266],
          ...,
          [ 1.1089, -0.7747, -0.0354,  ...,  2.4645, -0.4377, -1.3081],
          [ 1.0604, -0.5641, -0.7406,  ...,  2.7732, -0.3431, -1.1994],
          [ 0.8389, -0.3158, -1.3036,  ...,  3.0228, -0.2802, -0.8711]],
 
         [[ 1.0411,  0.0592, -0.3752,  ...,  2.2531, -0.9834,  0.1631],
          [ 0.0884,  3.3004,  0.6958,  ...,  1.4224,  0.3608,  0.0968],
          [ 0.3378,  1.2210,  0.1548,  ..., -0.0409, -0.1516, -2.5266],
          ...,
          [ 1.1089, -0.7747, -0.0354,  ...,  2.4645, -0.4377, -1.3081],
          [ 1.0604, -0.5641, -0.7406,  ...,  2.7732, -0.3431, -1.1994],
          [ 0.8389, -0.3158, -1.3036,  ...,  3.0228, -0.2802, -0.8711]]],
        device='cuda:0'),)

In [1]:
from models.gen_models import get_gen_model
from omegaconf import OmegaConf
import torchaudio
modelconfig = {
    'name': 'musicgen',
    'hparams': None
}
wav, sr = torchaudio.load('/home/lego/Gatech/Rupak/test.wav')
wav = torchaudio.functional.resample(wav, sr, 32000).cuda()
modelconfig= OmegaConf.create(modelconfig)
repr_extrat = get_gen_model(modelconfig).cuda()
info = repr_extrat(wav)


/home/lego/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/lego/.local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [ ]:
import os
import pickle
from tqdm import tqdm
import numpy as np

def process_directory(input_dir, output_base_dir):
    for filename in tqdm(os.listdir(input_dir)):
        if filename.endswith('.pkl'):
            input_path = os.path.join(input_dir, filename)
            
            with open(input_path, 'rb') as f:
                data = pickle.load(f)
            
            repr_data = data['repr']
            num_layers = repr_data.shape[0]
            
            for layer in range(num_layers):
                layer_data = repr_data[layer]  # Shape: [seq_len, dim]
                
                output_dir = os.path.join(output_base_dir, str(layer))
                os.makedirs(output_dir, exist_ok=True)
                
                output_path = os.path.join(output_dir, filename)
                
                with open(output_path, 'wb') as f:
                    pickle.dump({'repr': layer_data, 'meta': data['meta']}, f)

def main():
    base_input_dir = '/home/hice1/ywu3038/scratch/GTZAN/MusicGenSmall'
    base_output_dir = '/home/hice1/ywu3038/scratch/GTZAN/MusicGenSmall'
    
    for split in ['train', 'valid', 'test']:
        print(f"Processing {split} directory...")
        input_dir = os.path.join(base_input_dir, split)
        output_dir = os.path.join(base_output_dir, split)
        process_directory(input_dir, output_dir)
    
    print("Processing complete!")

if __name__ == '__main__':
    main()

: 